In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/GAN_for_Neural_Graph/ADHD/compared_models/baselines"

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['split_ids.json',
 'ADHD_Logistic_regression&SVM.ipynb',
 'Experiment_result.xlsx',
 'ADHD_data']

#Config

In [2]:
weight_threshold = 0.1
train_size = 700
val_size = 183

# Preprocessing


In [3]:
import os, sys
from tqdm import tqdm
import pickle
import json
import pandas as pd
import numpy as np
import random
from sklearn.utils import shuffle
from keras.utils import to_categorical
import sklearn
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [4]:
train_data = np.load('../../Data/train_data.npy')
test_data = np.load('../../Data/test_data.npy')
train_combine = np.load('../../Data/train_combine.npy')
test_combine = np.load('../../Data/test_combine.npy')

print(train_data.shape, train_combine.shape)
print(test_data.shape, test_combine.shape)

(758, 190, 190) (758, 3)
(171, 190, 190) (171, 3)


In [5]:
# shuffle
index = [i for i in range(train_data.shape[0])]
random.shuffle(index)
train_data = train_data[index]
train_combine = train_combine[index]

test_data, val_data, test_combine, val_combine = train_test_split(test_data, test_combine, test_size=0.5, shuffle=True)

print(train_data.shape, train_combine.shape)
print(test_data.shape, test_combine.shape)
print(val_data.shape, val_combine.shape)
print(train_combine[:10])

(758, 190, 190) (758, 3)
(85, 190, 190) (85, 3)
(86, 190, 190) (86, 3)
[[1.         0.51615355 1.        ]
 [0.         0.53933865 0.        ]
 [1.         0.29773217 0.        ]
 [0.         0.76168757 0.        ]
 [2.         0.60509312 0.        ]
 [2.         0.36374002 0.        ]
 [0.         0.35233751 0.        ]
 [0.         0.30064614 0.        ]
 [0.         0.30723426 0.        ]
 [0.         0.76320791 0.        ]]


In [6]:
np.save("ADHD_data/train_data.npy", train_data)
np.save("ADHD_data/test_data.npy", test_data)
np.save("ADHD_data/val_data.npy", val_data)

np.save("ADHD_data/train_combine.npy", train_combine)
np.save("ADHD_data/test_combine.npy", test_combine)
np.save("ADHD_data/val_combine.npy", val_combine)

In [ ]:
train_data_flatten = train_data.reshape(758, -1)
test_data_flatten = test_data.reshape(85, -1)
val_data_flatten = val_data.reshape(86, -1)

train_classes = train_combine[:, 0]
test_classes = test_combine[:, 0]
val_classes = val_combine[:, 0]

train_genders = to_categorical(train_combine[:, 2])
test_genders = to_categorical(test_combine[:, 2])
val_genders = to_categorical(val_combine[:, 2])

train_ages = np.expand_dims(train_combine[:, 1], axis=-1)
test_ages = np.expand_dims(test_combine[:, 1], axis=-1)
val_ages = np.expand_dims(val_combine[:, 1], axis=-1)

train_X = np.concatenate((train_data_flatten, train_genders, train_ages), axis = 1)
test_X = np.concatenate((test_data_flatten, test_genders, test_ages), axis = 1)
val_X = np.concatenate((val_data_flatten, val_genders, val_ages), axis = 1)

test_Y = to_categorical(test_classes)
print(test_Y.shape)

print(train_X.shape, train_classes.shape)
print(test_X.shape, test_classes.shape)
print(val_X.shape, val_classes.shape)

(85, 3)
(758, 36103) (758,)
(85, 36103) (85,)
(86, 36103) (86,)


# Train logistic regression

In [ ]:
from sklearn import linear_model
print('Training....')
clf_l = linear_model.LogisticRegression(max_iter=1000)
clf_l.fit(train_X, train_classes)
print('Finished')

Training....
Finished


# Test logistic regression

In [ ]:
labels_test = test_classes

# Evaluate SVM
labels_pred = clf_l.predict(test_X)
correct = np.sum(labels_pred==labels_test)
num_test = labels_pred.shape
three_c_acc = correct / num_test * 100

# classify 2 as 1
labels_2_test, labels_2_pred = labels_test.copy(), labels_pred.copy()
labels_2_test[labels_2_test==2] = 1
labels_2_pred[labels_2_pred==2] = 1

# calculate the accuracy
correct = np.sum(labels_2_test==labels_2_pred)
two_c_acc = correct / num_test * 100
print('three class acc: %.3f, two class acc: %.3f' % (three_c_acc, two_c_acc))
print("="*100)

# three class confusion metrics
target_names = ['class 0', 'class 1', 'class 2']
print('three class:')

# calculate precision, recall, f1 score
print(classification_report(labels_test, labels_pred, target_names=target_names, digits=4))

# calculate AUC
pred_y = np.zeros((labels_pred.shape[0], 3))
for i, label in enumerate(labels_pred):
    pred_y[i, int(label)] = 1
auc = sklearn.metrics.roc_auc_score(test_Y, pred_y)
print('AUC', auc)
print("="*100)

# two class confusion metrics
target_names = ['class 0', 'class 1']
print('two class:')
# calculate precision, recall, f1 score
print(classification_report(labels_2_test, labels_2_pred, target_names=target_names, digits=4))

# calcuate specificity
tn, fp, fn, tp = confusion_matrix(labels_2_test, labels_2_pred).ravel()
specificity = tn / (tn+fp)
print('specificity:', specificity)

# calculate AUC
pred_y = np.zeros((labels_2_pred.shape[0], 2))
for i, label in enumerate(labels_2_pred):
    pred_y[i, int(label)] = 1
auc = sklearn.metrics.roc_auc_score(labels_2_test, pred_y[:, 1])
print('AUC', auc)
print("="*100)

three class acc: 55.294, two class acc: 65.882
three class:
              precision    recall  f1-score   support

     class 0     0.6364    0.7955    0.7071        44
     class 1     0.4444    0.3077    0.3636        26
     class 2     0.3333    0.2667    0.2963        15

    accuracy                         0.5529        85
   macro avg     0.4714    0.4566    0.4557        85
weighted avg     0.5242    0.5529    0.5295        85

AUC 0.5997052336758827
two class:
              precision    recall  f1-score   support

     class 0     0.6364    0.7955    0.7071        44
     class 1     0.7000    0.5122    0.5915        41

    accuracy                         0.6588        85
   macro avg     0.6682    0.6538    0.6493        85
weighted avg     0.6671    0.6588    0.6513        85

specificity: 0.7954545454545454
AUC 0.6538248337028825


# Train SVM

In [ ]:
from sklearn import svm

print('Training....')
clf = svm.SVC()
clf.fit(train_X, train_classes)
print('Finished')

Training....
Finished


# Test SVM

In [ ]:
labels_test = test_classes

# Evaluate SVM
labels_pred = clf.predict(test_X)
correct = np.sum(labels_pred==labels_test)
num_test = labels_pred.shape
three_c_acc = correct / num_test * 100

# classify 2 as 1
labels_2_test, labels_2_pred = labels_test.copy(), labels_pred.copy()
labels_2_test[labels_2_test==2] = 1
labels_2_pred[labels_2_pred==2] = 1

# calculate the accuracy
correct = np.sum(labels_2_test==labels_2_pred)
two_c_acc = correct / num_test * 100
print('three class acc: %.3f, two class acc: %.3f' % (three_c_acc, two_c_acc))
print("="*100)

# three class confusion metrics
target_names = ['class 0', 'class 1', 'class 2']
print('three class:')

# calculate precision, recall, f1 score
print(classification_report(labels_test, labels_pred, target_names=target_names, digits=4))

# calculate AUC
pred_y = np.zeros((labels_pred.shape[0], 3))
for i, label in enumerate(labels_pred):
    pred_y[i, int(label)] = 1
auc = sklearn.metrics.roc_auc_score(test_Y, pred_y)
print('AUC', auc)
print("="*100)

# two class confusion metrics
target_names = ['class 0', 'class 1']
print('two class:')
# calculate precision, recall, f1 score
print(classification_report(labels_2_test, labels_2_pred, target_names=target_names, digits=4))

# calcuate specificity
tn, fp, fn, tp = confusion_matrix(labels_2_test, labels_2_pred).ravel()
specificity = tn / (tn+fp)
print('specificity:', specificity)

# calculate AUC
pred_y = np.zeros((labels_2_pred.shape[0], 2))
for i, label in enumerate(labels_2_pred):
    pred_y[i, int(label)] = 1
auc = sklearn.metrics.roc_auc_score(labels_2_test, pred_y[:, 1])
print('AUC', auc)
print("="*100)

three class acc: 51.765, two class acc: 51.765
three class:
              precision    recall  f1-score   support

     class 0     0.5176    1.0000    0.6822        44
     class 1     0.0000    0.0000    0.0000        26
     class 2     0.0000    0.0000    0.0000        15

    accuracy                         0.5176        85
   macro avg     0.1725    0.3333    0.2274        85
weighted avg     0.2680    0.5176    0.3531        85

AUC 0.5
two class:
              precision    recall  f1-score   support

     class 0     0.5176    1.0000    0.6822        44
     class 1     0.0000    0.0000    0.0000        41

    accuracy                         0.5176        85
   macro avg     0.2588    0.5000    0.3411        85
weighted avg     0.2680    0.5176    0.3531        85

specificity: 1.0
AUC 0.5


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
